# Parsing

Prior to entry in the model, the code will be parsed into a dependency structure. We do this to maximize the value of each 1-1 sample of code, since ordering of independent clauses is arbitrary.

In [10]:
import esprima
import networkx as nx

def parse_js_code(js_code):
    return esprima.parseScript(js_code, loc=True)

def create_dependency_graph(ast):
    G = nx.DiGraph()
    variables = {}
    
    # Traverse AST to identify variable declarations and their dependencies
    for statement in ast.body:
        if statement.type == 'VariableDeclaration':
            for declaration in statement.declarations:
                var_name = declaration.id.name
                var_line = declaration.loc.start.line
                if declaration.init:
                    dependencies = find_dependencies(declaration.init)
                    G.add_node(var_line, label=var_name)
                    for dep in dependencies:
                        if dep in variables:
                            G.add_edge(variables[dep], var_line)
                else:
                    G.add_node(var_line, label=var_name)
                variables[var_name] = var_line
    return G

def find_dependencies(expression):
    dependencies = []
    if expression.type == 'BinaryExpression':
        dependencies += find_dependencies(expression.left)
        dependencies += find_dependencies(expression.right)
    elif expression.type == 'Identifier':
        dependencies.append(expression.name)
    elif expression.type == 'CallExpression':
        if expression.callee.type == 'Identifier':
            dependencies.append(expression.callee.name)
        for arg in expression.arguments:
            dependencies += find_dependencies(arg)
    return dependencies

def print_graph(G):
    print("Nodes:")
    for node, data in G.nodes(data=True):
        print(f"Line {node}: {data['label']}")
    print("\nEdges:")
    for u, v in G.edges():
        print(f"Line {u} -> Line {v}")

js_code = """
var x = 1;
var y = x + 1;
var z = 12;
var sink = x + y + z;
"""

js_ast = parse_js_code(js_code)
graph = create_dependency_graph(js_ast)
print_graph(graph)


Nodes:
Line 2: x
Line 3: y
Line 4: z
Line 5: sink

Edges:
Line 2 -> Line 3
Line 2 -> Line 5
Line 3 -> Line 5
Line 4 -> Line 5


In [11]:
def all_topological_sorts(G):
    return [nodes for nodes in nx.all_topological_sorts(G)]

all_topological_sorts(graph)

[[4, 2, 3, 5], [2, 3, 4, 5], [2, 4, 3, 5]]

In [14]:
def collapse_sccs(G):
    # for each strongly connected component, collapse it into a single node whose value is all the strings of each node concatenated
    sccs = nx.strongly_connected_components(G)
    for scc in sccs:
        if len(scc) > 1:
            new_node = ''.join([G.nodes[node]['label'] for node in scc])
            G.add_node(new_node)
            G.remove_nodes_from(scc)
            for pred in G.predecessors(scc.pop()):
                G.add_edge(pred, new_node)
    return G

print_graph(collapse_sccs(graph))

Nodes:
Line 2: x
Line 3: y
Line 4: z
Line 5: sink

Edges:
Line 2 -> Line 3
Line 2 -> Line 5
Line 3 -> Line 5
Line 4 -> Line 5
